# AppBuilder-Knowledge：生产环境的知识库/文档/切片管理教学

[知识库组件](https://github.com/baidubce/app-builder/blob/master/docs/BasisModule/Platform/KnowledgeBase/knowledgebase.md)

知识库组件（KnowledgeBase）是对线上知识库操作的组件，可以通过SDK实现创建知识库、添加知识文档、查询知识库文档、删除知识文档等操作，可在平台console中查看结果。对console端知识库进行操作，可以通过SDK实现创建知识库、添加知识文档、查询知识库文档、删除知识文档等操作，可在平台console中查看结果

## 1. 创建Agent应用

### 1.1 平台Console创建Agent应用

- 创建空应用并发布

![](https://bj.bcebos.com/v1/appbuilder-sdk-components/console%E5%88%9B%E5%BB%BA%E6%B0%91%E6%B3%95%E5%85%B8.png?authorization=bce-auth-v1%2FALTAKGa8m4qCUasgoljdEDAzLm%2F2024-12-17T07%3A38%3A28Z%2F-1%2Fhost%2Ff497d9db3d8945859043fe24fcca4490c3e93a27042e42abc280d0fea7fd3305)

### 1.2 使用SDK调用创建的民法典智能问答Agent


In [ ]:
import appbuilder
import os

# 更换为自己的Appbuilder-token
os.environ["APPBUILDER_TOKEN"] = ""

# 更换为自己的App ID
app_id = ""
client = appbuilder.AppBuilderClient(app_id)
conversation_id = client.create_conversation()
message = client.run(conversation_id, "请输出民法典第二编第一章的内容")
# 打印对话结果
print(message.content.answer)

民法典第二编第一章的内容是关于物权编通则的一般规定，主要包括以下几个方面：

### 一、物权编的调整范围

* **第二百零五条**：本编调整因物的归属和利用产生的民事关系。

### 二、国家基本经济制度

* **第二百零六条**：国家坚持和完善公有制为主体、多种所有制经济共同发展，按劳分配为主体、多种分配方式并存，社会主义市场经济体制等社会主义基本经济制度。国家巩固和发展公有制经济，鼓励、支持和引导非公有制经济的发展。国家实行社会主义市场经济，保障一切市场主体的平等法律地位和发展权利。

### 三、物权保护原则

* **第二百零七条**：国家、集体、私人的物权和其他权利人的物权受法律平等保护，任何组织或者个人不得侵犯。

### 四、物权设立、变更、转让和消灭的基本原则

* **第二百零八条**：不动产物权的设立、变更、转让和消灭，应当依照法律规定登记。动产物权的设立和转让，应当依照法律规定交付。

### 五、其他规定

民法典第二编第一章还包含了一些其他与物权相关的一般性规定，这些规定为物权编后续章节的具体内容提供了基础和指导。

综上所述，民法典第二编第一章作为物权编通则的一般规定，明确了物权编的调整范围、国家基本经济制度、物权保护原则以及物权设立、变更、转让和消灭的基本原则，为后续章节的具体规定提供了基础和指导。


![](https://bj.bcebos.com/v1/appbuilder-sdk-components/%E6%B0%91%E6%B3%95%E5%85%B8%E7%AC%AC%E4%BA%8C%E7%BC%96%E7%AC%AC%E4%B8%80%E7%AB%A0%E4%B8%80%E8%88%AC%E8%A7%84%E5%AE%9A.png?authorization=bce-auth-v1%2FALTAKGa8m4qCUasgoljdEDAzLm%2F2024-12-17T08%3A10%3A14Z%2F-1%2Fhost%2F7852a4156383383e6190435f76b6709409def221e3835369f7d155d3c9dd438a)

我们可以观察到上述Agent基本回答除了对于民法典的第二编第一章一般规定的内容，但是对照上图中的内容不够准确，这说明LLM在运行中出现了一定程度的幻觉，所以我们需要使用RAG(KnowledgeBase)对Agent进行优化。

## 2. RAG(KnowledgeBase)

Appbuilder中支持多种实现创建知识库、添加知识文档、查询知识库文档、删除知识文档等操作方式，包括：
- 平台Console实现知识库操作
- 使用SDK/API代码态实现知识库操作
这里我们选择使用代码态SDK实现知识库的操作

### 2.1 创建知识库

首先我们需要创建知识库，这里我们使用SDK代码态实现知识库的创建

In [80]:
knowledge = appbuilder.KnowledgeBase()
resp = knowledge.create_knowledge_base(
        name="中华人民共和国民法典",
        description="中华人民共和国民法典的中文版",
        type="public",
    )
my_knowledge_base_id = resp.id # 传入知识库ID
my_knowledge = appbuilder.KnowledgeBase(my_knowledge_base_id)
print("知识库ID: ", my_knowledge.knowledge_id)

知识库ID:  6ac10164-e80a-4497-ae9f-06ac35f38b97


- 我们可以到平台Console中查看知识库是否创建成功，我们可以观察到知识库的名称和ID，知识库创建成功！

![](https://bj.bcebos.com/v1/appbuilder-sdk-components/%E6%9F%A5%E7%9C%8B%E7%9F%A5%E8%AF%86%E5%BA%93%E6%B0%91%E6%B3%95%E5%85%B8.png?authorization=bce-auth-v1%2FALTAKGa8m4qCUasgoljdEDAzLm%2F2024-12-17T08%3A37%3A16Z%2F-1%2Fhost%2F40c0d55c6d01d538853bf3b8167c9c5271d74083a20afd5d17590986b44eb954)


### 2.2 上传文档到知识库

上传文档到知识库共有以下三种方式：
- 上传文档到知识库
- 上传通用文档

##### 2.2.1 上传文档到知识库
- 主要提供自定义文档处理策略，向知识库添加文档
  - 文档格式：rawText (允许配置后续分割策略)
  - 文档处理策略

```python
knowledge_base_id = my_knowledge.knowledge_id
knowledge.create_documents(
	id=knowledge_base_id,
	contentFormat="rawText",
	source=appbuilder.DocumentSource(
		type="web",
		urls=["网页版的文档链接地址"],
		urlDepth=1,
	),
	processOption=appbuilder.DocumentProcessOption(
		template="custom",
		parser=appbuilder.DocumentChoices(
			choices=["layoutAnalysis", "ocr"]
		),
		chunker=appbuilder.DocumentChunker(
			choices=["separator"],
			separator=appbuilder.DocumentSeparator(
				separators=["。"],
				targetLength=300,
				overlapRate=0.25,
			),
			prependInfo=["title", "filename"],
		),
		knowledgeAugmentation=appbuilder.DocumentChoices(choices=["faq"]),
	),
)
```

#### 2.2.2 上传通用文档
- SDK支持基于代码态的文档上传方法`upload_file`,以及包含文档的自定义切分逻辑的向知识库添加文档的方法``add_document`
- 需要先将文档下载至本地

#### 采用上传通用文档的方案上传文档到知识库
- 下面将提供一个缺少第七编第十章的民法典txt文件，同时使用SDK将其上传到知识库中，首先下载我们储存到云端的txt文件。

In [81]:
# 下载云端民法典到本地
import requests

def download_file(url, local_filename):
    try:
        response = requests.get(url, stream=True)
        with open(local_filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

        print(f"文件已成功下载到 {local_filename}")
    except requests.RequestException as e:
        print(f"下载文件时发生错误: {e}")

# 使用示例
url = 'https://bj.bcebos.com/v1/appbuilder-sdk-components/mingfadian.txt?authorization=bce-auth-v1\
%2FALTAKGa8m4qCUasgoljdEDAzLm%2F2024-12-17T10%3A44%3A33Z%2F-1%2Fhost%2Fb97e44abe836f58de9632c374d4055c391a84f7998562493c22fdff8596cdf49'
local_filename = 'mingfadian.txt'  
download_file(url, local_filename)

文件已成功下载到 mingfadian.txt


- 接下来我们将下载好的mingfadian.txt文件使用SDK上传到Appbuilder的知识库中，使用`appbuilder.CustomProcessRule`设置文档的切分规则。

In [83]:
upload_res = knowledge.upload_file(local_filename)
add_res = knowledge.add_document(
    content_type="raw_text",
    file_ids=[upload_res.id],
    custom_process_rule=appbuilder.CustomProcessRule(
        separators=["?"], target_length=600, overlap_rate=0.3
    ),
    knowledge_base_id=my_knowledge.knowledge_id
)

- 此时就可以在平台Console中查看到我们上传的民法典知识库了

![](https://bj.bcebos.com/v1/appbuilder-sdk-components/%E6%B0%91%E6%B3%95%E5%85%B8%E7%9F%A5%E8%AF%86%E5%BA%93%E4%B8%AD%E6%9F%A5%E7%9C%8B%E5%AF%B9%E5%BA%94%E6%96%87%E4%BB%B6.png?authorization=bce-auth-v1%2FALTAKGa8m4qCUasgoljdEDAzLm%2F2024-12-17T08%3A51%3A35Z%2F-1%2Fhost%2F4025b0777017d0776e08ebe81c94b794c0883165ceaac1751d6c9824ed5727c2)

### 2.3 为Agent应用添加知识库

- 在平台console中找到我们创建的Agent应用，添加我们创建的民法典知识库，并更新发布应用

![](https://bj.bcebos.com/v1/appbuilder-sdk-components/%E5%BA%94%E7%94%A8%E6%B7%BB%E5%8A%A0%E7%9F%A5%E8%AF%86%E5%BA%93.png?authorization=bce-auth-v1%2FALTAKGa8m4qCUasgoljdEDAzLm%2F2024-12-17T08%3A55%3A22Z%2F-1%2Fhost%2F4213ddd932eb9db15a507d985092797f168f5bb338df0453ea0848a1f9814597)

- 再次调用我们的Agent应用，就可以看到我们添加民法典知识库之后，Agent应用可以正确回答我们的问题了

In [88]:
client = appbuilder.AppBuilderClient(app_id)
conversation_id = client.create_conversation()
message = client.run(conversation_id, "请输出民法典第二编第一章的内容")
# 打印对话结果
print(message.content.answer)

**民法典第二编第一章的内容如下**：

第一章　一般规定

第二百零五条　本编调整因物的归属和利用产生的民事关系。

第二百零六条　国家坚持和完善公有制为主体、多种所有制经济共同发展，按劳分配为主体、多种分配方式并存，社会主义市场经济体制等社会主义基本经济制度。国家巩固和发展公有制经济，鼓励、支持和引导非公有制经济的发展。国家实行社会主义市场经济，保障一切市场主体的平等法律地位和发展权利。

第二百零七条　国家、集体、私人的物权和其他权利人的物权受法律平等保护，任何组织或者个人不得侵犯。

第二百零八条　不动产物权的设立、变更、转让和消灭，应当依照法律规定登记。动产物权的设立和转让，应当依照法律规定交付。


- 可以观察到在使用了RAG(KnowledgeBase)之后，可以准确的输出【民法典的第二编第一分编第一章一般规定】部分的准确内容，接下来我们实验查询我们刻意删除掉的民法典第七编第十章部分的内容，查看Agent能否输出准确答案。(使用此方法模拟我们知识库中的文档更新之后，并没有被上传到知识库的情况)

In [89]:
client = appbuilder.AppBuilderClient(app_id)
conversation_id = client.create_conversation()
message = client.run(conversation_id, "请输出中华人民共和国民法典第七编第十章原文内容")
# 打印对话结果
print(message.content.answer)

无法回答此问题。


- 我们可以发现Agent并没有输出民法典的第七编第十章的准确内容，此时我们就需要Appbuilder的切片管理功能，对知识库新增切片

## 3. 知识库切片管理

Appbuilder中支持多种知识库切片管理操作方式，包括：
- 平台Console实现知识库切片管理
- 使用SDK/API代码态实现知识库切片管理(优势)
这里我们选择使用代码态SDK实现知识库切片管理的操作

### 3.1KnowledgeBase 代码态切片管理功能

* 创建切片
* 修改切片信息
* 获取切片信息
* 获取切片列表
* 删除切片


- 接下来我们使用SDK上传一个民法典第七编第十章部分的内容的切片

In [90]:
# 获取知识库全部文档:get_all_documents
doc_list = knowledge.get_all_documents(my_knowledge.knowledge_id)

# 获取doc_list列表的第一个文档的document_id
document_id = doc_list[0].id # 这里我们的知识库只有一个文档，所以获取第一个文档的document_id
my_knowledge = appbuilder.KnowledgeBase(my_knowledge_base_id)

# 这里我们使用来web_crawler函数来模拟爬虫功能，在生产环境中开发者可以使用自己的爬虫模块来周期性的获取最新的文档更新内容
def web_crawler():
    """
    此函数模拟爬虫功能，获取最新的文档更新内容内容
    """
    content = """
    中华人民共和国民法典第七编第十章原文内容如下:
    建筑物和物件损害责任
    第一千二百五十二条　建筑物、构筑物或者其他设施倒塌、塌陷造成他人损害的，由建设单位与施工单位承担连带责任，但是建设单位与施工单位能够证明不存在质量缺陷的除外。建设单位、施工单位赔偿后，有其他责任人的，有权向其他责任人追偿。
    因所有人、管理人、使用人或者第三人的原因，建筑物、构筑物或者其他设施倒塌、塌陷造成他人损害的，由所有人、管理人、使用人或者第三人承担侵权责任。
    第一千二百五十三条　建筑物、构筑物或者其他设施及其搁置物、悬挂物发生脱落、坠落造成他人损害，所有人、管理人或者使用人不能证明自己没有过错的，应当承担侵权责任。所有人、管理人或者使用人赔偿后，有其他责任人的，有权向其他责任人追偿。
    第一千二百五十四条　禁止从建筑物中抛掷物品。从建筑物中抛掷物品或者从建筑物上坠落的物品造成他人损害的，由侵权人依法承担侵权责任;经调查难以确定具体侵权人的，除能够证明自己不是侵权人的外，由可能加害的建筑物使用人给予补偿。可能加害的建筑物使用人补偿后，有权向侵权人追偿。
    物业服务企业等建筑物管理人应当采取必要的安全保障措施防止前款规定情形的发生;未采取必要的安全保障措施的，应当依法承担未履行安全保障义务的侵权责任。
    发生本条第一款规定的情形的，公安等机关应当依法及时调查，查清责任人。
    第一千二百五十五条　堆放物倒塌、滚落或者滑落造成他人损害，堆放人不能证明自己没有过错的，应当承担侵权责任。
    第一千二百五十六条　在公共道路上堆放、倾倒、遗撒妨碍通行的物品造成他人损害的，由行为人承担侵权责任。公共道路管理人不能证明已经尽到清理、防护、警示等义务的，应当承担相应的责任。
    第一千二百五十七条　因林木折断、倾倒或者果实坠落等造成他人损害，林木的所有人或者管理人不能证明自己没有过错的，应当承担侵权责任。
    第一千二百五十八条　在公共场所或者道路上挖掘、修缮安装地下设施等造成他人损害，施工人不能证明已经设置明显标志和采取安全措施的，应当承担侵权责任。
    窨井等地下设施造成他人损害，管理人不能证明尽到管理职责的，应当承担侵权责任。
    """
    return content


resp = my_knowledge.create_chunk(documentId=document_id, content=web_crawler())
print("切片ID: ", resp.id)

切片ID:  f1e1962f-70dd-4573-9068-37485980ac5c


- 接下来我们再次调用Agent应用，查看一下他对民法典第七编第十章的说明。

In [ ]:
client = appbuilder.AppBuilderClient(app_id)
conversation_id = client.create_conversation()
message = client.run(conversation_id, "请输出中华人民共和国民法典第七编第十章原文内容")
# 打印对话结果
print(message.content.answer)

以下是《中华人民共和国民法典》第七编第十章的全部内容：

**建筑物和物件损害责任**

**第一千二百五十二条**　建筑物、构筑物或者其他设施倒塌、塌陷造成他人损害的，由建设单位与施工单位承担连带责任，但是建设单位与施工单位能够证明不存在质量缺陷的除外。建设单位、施工单位赔偿后，有其他责任人的，有权向其他责任人追偿。

因所有人、管理人、使用人或者第三人的原因，建筑物、构筑物或者其他设施倒塌、塌陷造成他人损害的，由所有人、管理人、使用人或者第三人承担侵权责任。

**第一千二百五十三条**　建筑物、构筑物或者其他设施及其搁置物、悬挂物发生脱落、坠落造成他人损害，所有人、管理人或者使用人不能证明自己没有过错的，应当承担侵权责任。所有人、管理人或者使用人赔偿后，有其他责任人的，有权向其他责任人追偿。

**第一千二百五十四条**　禁止从建筑物中抛掷物品。从建筑物中抛掷物品或者从建筑物上坠落的物品造成他人损害的，由侵权人依法承担侵权责任；经调查难以确定具体侵权人的，除能够证明自己不是侵权人的外，由可能加害的建筑物使用人给予补偿。可能加害的建筑物使用人补偿后，有权向侵权人追偿。

物业服务企业等建筑物管理人应当采取必要的安全保障措施防止前款规定情形的发生；未采取必要的安全保障措施的，应当依法承担未履行安全保障义务的侵权责任。

发生本条第一款规定的情形的，公安等机关应当依法及时调查，查清责任人。

**第一千二百五十五条**　堆放物倒塌、滚落或者滑落造成他人损害，堆放人不能证明自己没有过错的，应当承担侵权责任。

**第一千二百五十六条**　在公共道路上堆放、倾倒、遗撒妨碍通行的物品造成他人损害的，由行为人承担侵权责任。公共道路管理人不能证明已经尽到清理、防护、警示等义务的，应当承担相应的责任。

**第一千二百五十七条**　因林木折断、倾倒或者果实坠落等造成他人损害，林木的所有人或者管理人不能证明自己没有过错的，应当承担侵权责任。

**第一千二百五十八条**　在公共场所或者道路上挖掘、修缮安装地下设施等造成他人损害，施工人不能证明已经设置明显标志和采取安全措施的，应当承担侵权责任。

窨井等地下设施造成他人损害，管理人不能证明尽到管理职责的，应当承担侵权责任。


- 可以观察到此次Agent运行准确的的输出了民法典第七编第十章的内容

#### 其他的知识库文档&切片管理方法

###### 修改切片信息`modify_chunk`

- 如果我们对之前设置的切片信息不满意，可以通过`modify_chunk`方法进行修改。

###### 获取切片信息`describe_chunk`

- 我需要查看我刚才创建切片信息，可以使用`describe_chunk`方法

###### 删除切片信息`delete_chunk`

- 切片信息创建出来后，如果不需要了，可以通过`delete_chunk`方法进行删除。

##### 删除文档和知识库

- 当你不再需要文档知识库时，可以删除它，SDK同样提供删除文档和知识库的方法
  - 从知识库删除文档 `KnowledgeBase().delete_document`
  - 删除知识库`delete_knowledge_base`